## xGBoostTotalIncidents

*   Input:  
  1.   *trainTractsDroppedStandard.csv*  
  2.   *testTractsDroppedStandard.csv* 
*   Does:  
  1.   Trains an XGBoost Model on census tracts to predict
       total incident count per capita.
  2.   Plots features found to be important by the trained model. 
*   Output:
  1.   Plots showing feature importances extracted from the 
       trained model.

## Import Data

In [ ]:
# Install libraries
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split


# Load training data
X_train_full = pd.read_csv('data_source/trainTractsDroppedStandard.csv', 
                            engine='python'
                           )
X_train_full.set_index('TRACTA', inplace=True, drop=True)




# Load testing data
X_test_full = pd.read_csv('data_source/testTractsDroppedStandard.csv', 
                            engine='python'
                           )
X_test_full.set_index('TRACTA', inplace=True, drop=True)





# Rescale target (otherwise model will almost always predict 0.0)
X_train_full['incidents_per_capita_log'] = np.log(X_train_full['incidents_per_capita'])
X_test_full['incidents_per_capita_log'] = np.log(X_test_full['incidents_per_capita'])




# Get columns to drop
drop_cols1 = list(X_train_full.loc[:, 'total_pop':'RPL_THEMES'].columns)
drop_cols2 = ['Unnamed: 0',  'total_incidents', 'incidents_per_capita', 'medical_per_capita', 'fire_per_capita']
drop_cols = drop_cols1 + drop_cols2



# Drop unnecessary columns
X_train_full = X_train_full.drop(columns=drop_cols)
X_test_full = X_test_full.drop(columns=drop_cols)



# Rename columns for enhanced readability
rename_mapping = {'Oregon': 'Tract in Oregon',
                  'percent_male': 'Pct Male',
                  'percent_w_public_assist_income_exp0.3': 'Pct with Public Assist Income',
                  'percent_1.01to1.5_per_room_exp0.4': 'Pct Households with 1.01 - 1.5 per Room',
                  'percent_leave_for_work_4tomidnight_sqrt': 'Pct Leaving for Work after 4 PM',
                  'percent_5to14min_commute_sqrt': 'Pct Having 5 - 14 Minute Commute',
                  'total_pop_sqrt': 'Total Population',
                  'percent_leave_for_work_before_6am_sqrt': 'Pct Leaving for Work before 6 AM',
                  'percent_15to24min_commute': 'Pct Having 14 - 24 Minute Commute',
                  'num_housing_units_boxcox': 'Num of Housing Units',
                  'median_year_built': 'Median Year When Buildings Built',
                  'percent_no_health_insur_sqrt': 'Pct with No Health Insurance',
                  'percent_renter_occ_sqrt': 'Pct Housing Renter-Occupied',
                  'percent_white_alone_exp1.4': 'Pct White',
                  'percent_over90min_commute_sqrt': 'Pct Commuting over 90 min',
                  'percent_multi_race_sqrt': 'Pct Multiracial',
                  'SPL_THEME4': 'Housing Type And Transportation Score',
                  'percent_0to19_years': 'Pct 0 - 19 Years Old',
                  'percent_4_person_hh': 'Pct of 4-Person Households',
                  'percent_drive_to_work_boxcox': 'Pct who Drive to Work',
                  'percent_asian_alone_exp0.3': 'Pct Asian',
                  'SPL_THEME2': 'Household Composition and Disability Score',
                  'percent_black_alone_exp0.3': 'Pct Black',
                  'percent_medicaid_only_sqrt': 'Pct with Only Medicaid',
                  'percent_3_person_hh': 'Pct of 3-Person Households',
                  'percent_employer_only': 'Pct with Only Employer-provided Health Insurance',
                  'percent_1.51to2_per_room_exp0.3': 'Pct Households with 1.51 - 2 per Room',
                  'percent_public_transpo_to_work_exp0.4': 'Pct Commuting with Public Transport',
                  'EP_GROUPQ_exp0.2': 'Pct Persons in Group Quarters',
                  'percent_vacant_exp0.2': 'Pct Vacant Houses',
                  'percent_ue_in_labor_force_sqrt': 'Pct Unemployed in Labor Force',
                  'percent_25to34min_commute': 'Pct Having 25 - 34 Minute Commute',
                  'percent_tricare_only_exp0.7': 'Pct with Tricare Health Insurance', 
                  'Idaho': 'Tract in Idaho',
                  'percent_VA_only_exp0.3': 'Pct with VA Health Insurance',
                  'percent_2_person_hh': 'Pct of 2-Person Households',
                  'percent_1.85to1.99_IPL_sqrt': 'Pct of Incomes 1.85 to 1.99 IPL'}
X_train_full.rename(columns=rename_mapping, inplace=True)
X_test_full.rename(columns=rename_mapping, inplace=True)



# Allow us to see all columns of our dataframe
pd.set_option('max_columns', None)

## Get Data in Proper Shape for Predictions

In [ ]:
# Separate target from predictors
y_train = X_train_full['incidents_per_capita_log']
X_train = X_train_full.drop(columns='incidents_per_capita_log')

# Separate target from predictors
y_test = X_test_full['incidents_per_capita_log']
X_test = X_test_full.drop(columns='incidents_per_capita_log')

## Modeling

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from xgboost import XGBRegressor

# Define our model
fixed_params = {'objective': 'reg:squarederror', 
                'random_state': 42,
                'n_jobs': 1
               }
param_dist = {'max_depth': Integer(1,3),
              'learning_rate': Real(0.01, 0.2),
              'min_child_weight': Integer(0, 10),
              'max_delta_step': Integer(0, 20),
              'colsample_bytree': Real(0.01, 1.0, 'uniform'),
              'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
              'n_estimators': Integer(100, 1000),
              'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
              'reg_alpha': Real(0.0, 2.0),
              'reg_lambda': Real(0.0, 8.0),
             }
regressor = XGBRegressor(**fixed_params)

search = BayesSearchCV(regressor,
                    param_dist,
                    scoring='neg_mean_absolute_error',
                    cv=5,
                    n_iter=75,
                    n_jobs=-1,
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=0)

search.fit(X_train, y_train)

In [ ]:
print('Best parameters:  ', search.best_params_)
print('Best MAE score:  ', search.best_score_)
best_estimator = search.best_estimator_

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = best_estimator.predict(X_test)
mae = mean_absolute_error(predictions, y_test) # Your code here
print(mae)

## Model Explainability

In [ ]:
import shap

# Code to fix a bug that prevents SHAP from interacting with the XGBoost library [3]
mybooster = best_estimator.get_booster()    
model_bytearray = mybooster.save_raw()[4:]
def myfun(self=None):
    return model_bytearray
mybooster.save_raw = myfun

# Get SHAP values
explainer = shap.TreeExplainer(mybooster)
shap_values = explainer.shap_values(X_train)

In [ ]:
# Plot absolute feature importance of our XGBoost Model
plt.figure()
plt.title('Average Feature Contribution Towards Total Incidents per Capita', pad=20, fontsize=19, fontweight='bold')
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
# Get a summary plot showing groupings of data and correlations between features and the target variable.
plt.figure()
plt.title('Summary Plot of Our XGBoost Model', pad=25, fontsize=20, fontweight='bold')
shap.summary_plot(shap_values, X_train)

In [ ]:
# Visualize the explanation for an arbitrary prediction (use matplotlib=True to avoid using Javascript)
shap.force_plot(explainer.expected_value, shap_values[23,:], X_train.iloc[23,:], matplotlib=True)

In [ ]:
# Create plot showing effect of 'Percent Commuting over 90 min' on model output
medIncome_dependencePlot = shap.dependence_plot('Pct Commuting over 90 min', shap_values, X_train, show=False)